In [1]:
import matplotlib.image as im
import json
import pandas as pd
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.image as im
import json
import pandas as pd
import os
import json
import torch
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import *
from transformers import ViTImageProcessor, ViTModel
from sentence_transformers import SentenceTransformer


d:\CONDA_ENV\deep_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\CONDA_ENV\deep_learning\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
d:\CONDA_ENV\deep_learning\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


# Metodi Gestione Immagini

In [4]:
def apri(file_path):
    # Definisci il percorso del file JSONL


    # Lista per memorizzare i record JSON analizzati
    json_records = []

    # Apri il file JSONL in modalità lettura
    with open(file_path, "r") as file:
        # Leggi il file riga per riga
        for line in file:
            # Analizza il record JSON della riga corrente e lo aggiungi alla lista
            json_record = json.loads(line.strip())
            json_records.append(json_record)

    return pd.DataFrame(json_records)

In [5]:
train = apri("train.jsonl")

train_pos = train[train["label"] == 0].copy()
train_neg = train[train["label"] == 1].copy()

n_pos = len(train_pos)
n_neg = len(train_neg)

n = n_pos
if(n_neg < n_pos):
    n= n_neg

result = pd.concat([train_pos[0:n], train_neg[0:n]])
result.to_json('train_ridotto.jsonl', orient='records', lines=True)

# FUSION AND CLASSIFICATION

In [2]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model_img = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
#model_img.requires_grad_= False
model_text = SentenceTransformer('all-mpnet-base-v2')
#model_text.requires_grad_= False
print("Modelli Caricati")


model_img.requires_grad_= True
model_text.requires_grad_= True

loading configuration file preprocessor_config.json from cache at C:\Users\fulvi/.cache\huggingface\hub\models--google--vit-base-patch16-224-in21k\snapshots\7cbdb7ee3a6bcdf99dae654893f66519c480a0f8\preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file config.json from cache at C:\Users\fulvi/.cache\huggingface\hub\models--google--vit-base-patch16-224-in21k\snapshots\7cbdb7ee3a6bcdf99dae654893f66519c480a0f8\config.json
Model confi

Modelli Caricati


In [7]:
train = apri("train.jsonl")


text = train['text'].tolist()
img =  train.iloc[0]["img"]
img = im.imread(f"image/{img}")
print(img.shape)
img = processor(images=img, return_tensors="pt")
img = img['pixel_values']
print(img.shape)
print(len(text))

img_emb = model_img(img)
text_emb = model_text.encode(text[0])

FileNotFoundError: [Errno 2] No such file or directory: 'image/img/42953.png'

In [ ]:
print(f"Size dell'embedding dell'immagine: {img_emb.pooler_output.squeeze().detach().numpy().shape}")
print(f"Size dell'embedding del testo: {text_emb.shape}")

Size dell'embedding dell'immagine: (768,)
Size dell'embedding del testo: (768,)


#### DATALOADER:

In [3]:


class MemeDataset(Dataset):
    def __init__(self, data_path, image_folder, transform=None):
        self.data = []
        self.image_folder = image_folder
        self.transform = transform

        # Carica i dati dal file JSON
        with open(data_path, "r") as f:
            for line in f:
                entry = json.loads(line)
                self.data.append(entry)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        img_name = os.path.join(self.image_folder, entry["img"])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        text = entry["text"]
        label = entry["label"]

        return image, text, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensiona l'immagine a 224x224 pixel
    transforms.ToTensor(),          # Converti l'immagine in un tensore
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizza i valori dei pixel
])


#### MODEL:

In [4]:
class MemeClassifier(nn.Module):
    def __init__(self, size_emb, model_img, model_text, device):
        super(MemeClassifier, self).__init__()
        self.model_img = model_img
        self.model_text = model_text
        self.device = device
        
        # self.img_weight = nn.Linear(size_emb, size_emb)
        # self.text_weight = nn.Linear(size_emb, size_emb)
        
        self.fusion1_fc = nn.Linear(2*size_emb, 1024)
        self.fusion2_fc = nn.Linear(1024, 512)
        self.fusion3_fc = nn.Linear(512,128)
        self.fusion4_fc = nn.Linear(128,64)
        self.output_fc = nn.Linear(64, 1)


    def forward(self, image, text):
        image_embedding = self.model_img(image).pooler_output
        #print("image embedding shape: ", image_embedding.shape)
        text_embedding = self.model_text.encode(text)
        text_embedding_tensor = torch.from_numpy(text_embedding).to(self.device)
        #print("text embedding shape: ", text_embedding_tensor.shape)
        # img_weight  = self.img_weight(image_embedding)
        # text_weight = self.text_weight(text_embedding_tensor)
        # emb = img_weight + text_weight
        emb = torch.cat((image_embedding, text_embedding_tensor), 1)
        #print("fused emb shape: ", emb.shape)
        fused_hidden = nn.ReLU()(self.fusion1_fc(emb))
        fused_hidden = nn.ReLU()(self.fusion2_fc(fused_hidden))
        fused_hidden = nn.ReLU()(self.fusion3_fc(fused_hidden))
        fused_hidden = nn.ReLU()(self.fusion4_fc(fused_hidden))
        output = nn.Sigmoid()(self.output_fc(fused_hidden))
        return output #, img_weight, text_weight


#### TRAIN LOOP:

In [5]:
def train_model(model, train_dataloader, criterion, optimizer, epochs, device, model_file):
    model.train()
    min_loss = float("inf")
    stop = False
    for epoch in range(epochs):
        epoch_loss = 0.0
        # for batch in train_dataloader:
        pbar = tqdm(train_dataloader)
        for i, (images, texts, labels) in enumerate(pbar):
            images = images.to(device)
            # texts = texts.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images, texts)

            if np.isnan(outputs.cpu().detach().numpy()[:,0][0]):
                print(f"({texts})")
                #print(f"({np.min(im_emb)},{np.max(im_emb)})")
                #print(f"({np.min(text_emb)},{np.max(text_emb)})")
                print(f"{outputs})")
                print(f"({labels})")
                #print(f"{outputs})")
                stop = True

            else:
                loss = criterion(outputs.float(), labels.unsqueeze(1).float())
                loss.backward()
                optimizer.step()
                #total_loss += loss.item()
                pbar.set_postfix(BCE=loss.item())
                epoch_loss += loss.item()
                stop = False
        if stop:
            break
        
        epoch_loss /= len(train_dataloader)
        if epoch_loss < min_loss :
            min_loss = epoch_loss
            print(f"Loss at epoch {epoch + 1}: {epoch_loss:.3f}")
            torch.save(model.state_dict(), f"{model_file}.pt")



In [6]:
# Iperparametri
size_emb = 768
learning_rate = 1e-4
epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_file = './results/vit_mpnet_6100_32_new'
data_path = "train.jsonl"
image_folder = "data/"
batch_size = 4 #16

#### RUN TRAINING:

In [11]:
dataset = MemeDataset(data_path, image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = MemeClassifier(size_emb, model_img, model_text, device)
model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Addestramento del modello
train_model(model, dataloader, criterion, optimizer, epochs, device, model_file)

100%|██████████| 532/532 [02:52<00:00,  3.08it/s, BCE=0.644]


Loss at epoch 1: 0.647


100%|██████████| 532/532 [02:51<00:00,  3.10it/s, BCE=0.382]


Loss at epoch 2: 0.581


100%|██████████| 532/532 [02:49<00:00,  3.14it/s, BCE=0.611]


...

Loss at epoch 18: 0.057


100%|██████████| 532/532 [02:48<00:00,  3.16it/s, BCE=0.0432]  


Loss at epoch 19: 0.053


100%|██████████| 532/532 [02:50<00:00,  3.12it/s, BCE=0.00621] 


Loss at epoch 20: 0.046


In [14]:
# Eval
model = MemeClassifier(size_emb, model_img, model_text, device)
model.load_state_dict(torch.load(f"{model_file}.pt"))
model.to(device)
model.eval()

dataset_test = MemeDataset("dev.jsonl", "image/", transform=transform)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

pred_labels = []
labels = []
pbar = tqdm(dataloader_test)
for step, (image, text, label) in enumerate(pbar):

    image = image.to(device)
    #label = label.to(device)
    output = model(image, text)
    predictions = (output.cpu().detach().numpy() > 0.5).astype(int)
    pred_labels.extend(predictions)
    labels.extend(label)

print(classification_report(labels, pred_labels))

100%|██████████| 500/500 [00:16<00:00, 30.39it/s]

              precision    recall  f1-score   support

           0       0.55      0.67      0.61       250
           1       0.58      0.46      0.51       250

    accuracy                           0.56       500
   macro avg       0.57      0.56      0.56       500
weighted avg       0.57      0.56      0.56       500



#### RUN TRAINING CON NORMALIZZAZIONE:

In [ ]:
import torch.nn.functional as F

class MemeClassifier(nn.Module):
    def __init__(self, size_emb, model_img, model_text, device):
        super(MemeClassifier, self).__init__()
        self.model_img = model_img
        self.model_text = model_text
        self.device = device
        self.fusion1_fc = nn.Linear(2 * size_emb, 1024)
        self.fusion2_fc = nn.Linear(1024, 512)
        self.fusion3_fc = nn.Linear(512, 128)
        self.fusion4_fc = nn.Linear(128, 64)
        self.output_fc = nn.Linear(64, 1)

    def forward(self, image, text):
        image_embedding = self.model_img(image).pooler_output
        text_embedding = self.model_text.encode(text)
        text_embedding_tensor = torch.from_numpy(text_embedding).to(self.device)

        # Normalize the image and text embeddings
        image_embedding = F.normalize(image_embedding, p=2, dim=1)
        text_embedding_tensor = F.normalize(text_embedding_tensor, p=2, dim=1)

        # Concatenate the normalized embeddings
        emb = torch.cat((image_embedding, text_embedding_tensor), 1)

        fused_hidden = nn.ReLU()(self.fusion1_fc(emb))
        fused_hidden = nn.ReLU()(self.fusion2_fc(fused_hidden))
        fused_hidden = nn.ReLU()(self.fusion3_fc(fused_hidden))
        fused_hidden = nn.ReLU()(self.fusion4_fc(fused_hidden))
        output = nn.Sigmoid()(self.output_fc(fused_hidden))
        return output


In [ ]:
dataset = MemeDataset(data_path, image_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = MemeClassifier(size_emb, model_img, model_text, device)
model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Addestramento del modello
train_model(model, dataloader, criterion, optimizer, epochs, device, model_file)

100%|██████████| 2125/2125 [22:49<00:00,  1.55it/s, BCE=0.861]


Loss at epoch 1: 0.594


100%|██████████| 2125/2125 [19:10<00:00,  1.85it/s, BCE=0.633]


Loss at epoch 2: 0.552


100%|██████████| 2125/2125 [18:18<00:00,  1.93it/s, BCE=0.596] 


...

Loss at epoch 18: 0.288


100%|██████████| 2125/2125 [19:45<00:00,  1.79it/s, BCE=0.333]   


Loss at epoch 19: 0.274


100%|██████████| 2125/2125 [18:52<00:00,  1.88it/s, BCE=0.0154]  


Loss at epoch 20: 0.265


In [ ]:
# Eval
model = MemeClassifier(size_emb, model_img, model_text, device)
model.load_state_dict(torch.load(f"{model_file}.pt"))
model.to(device)
model.eval()

dataset_test = MemeDataset("dev.jsonl", "data/", transform=transform)
dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

pred_labels = []
labels = []
pbar = tqdm(dataloader_test)
for step, (image, text, label) in enumerate(pbar):

    image = image.to(device)
    #label = label.to(device)
    output = model(image, text)
    predictions = (output.cpu().detach().numpy() > 0.5).astype(int)
    pred_labels.extend(predictions)
    labels.extend(label)

print(classification_report(labels, pred_labels))

100%|██████████| 500/500 [00:52<00:00,  9.44it/s]

              precision    recall  f1-score   support

           0       0.54      0.79      0.64       250
           1       0.61      0.32      0.42       250

    accuracy                           0.56       500
   macro avg       0.57      0.56      0.53       500
weighted avg       0.57      0.56      0.53       500

